In [20]:
!pip install python-utils

In [18]:
jsonfile="/content/drive/MyDrive/Object Recognition Dataset/instances_val2017.json/instances_val2017.json"
datafolder="/content/drive/MyDrive/Object Recognition Dataset/val2017"

In [4]:
import os
import utils

In [2]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 27 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.69 MiB | 12.92 MiB/s, done.
Resolving deltas: 100% (11718/11718), done.


In [5]:
%cd yolov5/

/content/yolov5


In [6]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 23.6 MB/s eta 0:00:00


In [7]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['yolov5s.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CPU

100% 14.1M/14.1M [00:00<00:00, 88.2MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/2 /

In [ ]:
display = utils.notebook_init()
display.Image(filename='runs/detect/exp/bus.jpg', width=600)

In [16]:
%cd ..



/content


In [17]:
import os
os.makedirs("data")

In [ ]:
for datatype in ["train","test","val"]:
  for fold in ["images","labels"]:
    os.makedirs(f"data/{datatype}/{fold}")

In [ ]:
with open(jsonfile,"r") as f:
  jf=json.load(f)

In [ ]:
jf["annotations"][0].keys()

In [ ]:
jf["images"][0].keys()

In [ ]:
jf["categories"][0].keys()

In [ ]:
dd={}
for i in jf["images"]:
  dd[i["id"]]=[i["file_name"],i["height"],i["width"]]

In [ ]:

dic={}
for i in jf["annotations"]:
  if i["image_id"] not in dic:
    dic[i["image_id"]]={}
    dic[i["image_id"]]["bboxes"]=[]
    dic[i["image_id"]]["file_name"]=dd[i["image_id"]][0]
    dic[i["image_id"]]["height"]=dd[i["image_id"]][1]
    dic[i["image_id"]]["width"]=dd[i["image_id"]][2]
  dic[i["image_id"]]["bboxes"].append((i["category_id"],i["bbox"]))

In [ ]:
import shutil
len(dic)

In [ ]:
for i,id in enumerate(dic):
  width=dic[id]["width"]
  height=dic[id]["height"]
  if i<4000:
    name=dic[id]["file_name"].split(".")[0]
    src=datafolder+"/"+dic[id]["file_name"]


    dst=f"data/train/images/{name}.jpg"
    shutil.copyfile(src,dst)
    with open(f"data/train/labels/{name}.txt","w") as f:
      for j in dic[id]["bboxes"]:
        f.write(f"{j[0]} {(2*j[1][0]+j[1][2])/(2*width)} {(2*j[1][1]+j[1][3])/(2*height)} {j[1][2]/width} {j[1][3]/height} \n")
    if i>3000:
      dst1=f"data/val/images/{name}.jpg"
      shutil.copyfile(src,dst1)
      dst2=f"data/val/labels/{name}.txt"
      s2=f"data/train/labels/{name}.txt"
      shutil.copyfile(s2,dst2)
  else:
    name=dic[id]["file_name"].split(".")[0]
    src=datafolder+"/"+dic[id]["file_name"]
    dst=f"data/test/images/{name}.jpg"
    shutil.copyfile(src,dst)
    with open(f"data/test/labels/{name}.txt","w") as f:
      for j in dic[id]["bboxes"]:
        f.write(f"{j[0]} {(2*j[1][0]+j[1][2])/(2*width)} {(2*j[1][1]+j[1][3])/(2*height)} {j[1][2]/width} {j[1][3]/height}\n")
  print(i)

In [ ]:
cat={}
for i in jf["categories"]:
  cat[i["id"]]=i["name"]

In [ ]:
catlist=["" for i in range(91)]
for i in cat:
  catlist[i]=cat[i]

In [ ]:
with open("data.yaml","w") as f:
  f.write(f"train: ../data/train/images\nval: ../data/val/images\n\nnc: 91\nnames: {catlist}")

In [ ]:
import glob
TRAIN=True
def set_res_dir():
    # Directory to store results
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{res_dir_count}"
    return RES_DIR

In [ ]:
RES_DIR=set_res_dir()
EPOCHS=20
!python train.py --data ../data.yaml --weights yolov5s.pt --img 640 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}

In [ ]:
!python detect.py --weights runs/train/{RES_DIR}/weights/best.pt --img 640 --conf 0.85 --source ../data/test/images